In [72]:
#%pip install "flaml[automl]" openml

In [73]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from pandas.tseries.offsets import DateOffset
from itertools import product
from dateutil.relativedelta import relativedelta
from datetime import datetime
import plotly.graph_objects as go
#from flaml import AutoML
from sklearn.metrics import mean_squared_error, r2_score
import boto3
import os
import io

In [74]:
def download_og_from_s3():
    # Download original data from s3 for preprocessing and model training
    boto_session = boto3.session.Session(region_name='ap-northeast-1', profile_name='asurion-apac-nonprod.apacdev')
    s3_client = boto_session.client("s3")   
    
    # Load SCM_Demand_Singapore 1.csv
    obj = s3_client.get_object(
        Bucket="ai-ml-dev-dataset", Key="scm-forecast-data/dataset/SCM_Demand_Singapore 1.csv"
    )
    data = obj["Body"].read()
    imported_data_SG = pd.read_csv(io.BytesIO(data))

    # Load SCM_Demand_Malaysia_1.csv
    obj = s3_client.get_object(
        Bucket="ai-ml-dev-dataset", Key="scm-forecast-data/dataset/SCM_Demand_Malaysia_1.csv"
    )
    data = obj["Body"].read()
    imported_data_MY = pd.read_csv(io.BytesIO(data))

    # Load SCM_Demand_Thailand_2022_to_2023_1.csv
    obj = s3_client.get_object(
        Bucket="ai-ml-dev-dataset", Key="scm-forecast-data/dataset/SCM_Demand_Thailand_2022_to_2023_1.csv"
    )
    data = obj["Body"].read()
    imported_data_TH1 = pd.read_csv(io.BytesIO(data))

    # Load SCM_Demand_Thailand_2019_to_2021_1.csv
    obj = s3_client.get_object(
        Bucket="ai-ml-dev-dataset", Key="scm-forecast-data/dataset/SCM_Demand_Thailand_2019_to_2021_1.csv"
    )
    data = obj["Body"].read()
    imported_data_TH2 = pd.read_csv(io.BytesIO(data))

    # Concatenate all data
    imported_data = pd.concat([imported_data_MY,imported_data_SG,imported_data_TH1,imported_data_TH2])

    # Rename 'Shipped Claims' to 'Claims'
    imported_data = imported_data.rename(columns={"Shipped Claims": "Claims",})

    # Load Release_Details.csv
    obj = s3_client.get_object(
        Bucket="ai-ml-dev-dataset", Key="scm-forecast-data/dataset/Release_Details.csv"
    )
    data = obj["Body"].read()
    release_data = pd.read_csv(io.BytesIO(data))

    return imported_data, release_data

In [75]:
imported_data, release_data = download_og_from_s3()

/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/2044700602.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  imported_data_TH1 = pd.read_csv(io.BytesIO(data))


In [76]:
imported_data

,YearMonth,Country,Client,product,program,product_launch_date,Make,Phone_Series,Phone_launch_date,Model_Family,...,Shipped_Claim,Swap_Shipped_Claim,Replace_Shipped_Claim,LFL_Shipped_Claim,NLFL_Shipped_Claim,IR_Rate_Monthly,IR_Rate_Swap_Monthly,IR_Rate_Replace_Monthly,Churn_Rate_Monthly,Churn
0,201901,Malaysia,Celcom,Celcom_SwapOnly_Monthly,Celcom - Swap - Monthly,2015-11-24,Samsung,GALAXY A02,2018-01-01,SAMSUNG GALAXY A02S,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
1,201901,Malaysia,Celcom,Celcom_SwapOnly_Monthly,Celcom - Swap - Monthly,2015-11-24,Samsung,GALAXY A02,2018-01-01,SAMSUNG GALAXY A02S,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
2,201901,Malaysia,Celcom,Celcom_SwapOnly_Monthly,Celcom - Swap - Monthly,2015-11-24,Samsung,GALAXY A03,2022-01-01,SAMSUNG GALAXY A03,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
3,201901,Malaysia,Celcom,Celcom_SwapOnly_Monthly,Celcom - Swap - Monthly,2015-11-24,Samsung,GALAXY A03,2022-01-01,SAMSUNG GALAXY A03,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
4,201901,Malaysia,Celcom,Celcom_SwapOnly_Monthly,Celcom - Swap - Monthly,2015-11-24,Samsung,GALAXY A03,2018-01-01,SAMSUNG GALAXY A03S,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368118,202112,Thailand,True,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,PROTECH SWAP REPLACE MONTHLY,2020-08-19,VIVO,NaN,2019-11-01,VIVO V20 PRO,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
368119,202112,Thailand,True,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,PROTECH SWAP REPLACE MONTHLY,2020-08-19,VIVO,NaN,2019-11-01,VIVO V20 PRO,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
368120,202112,Thailand,True,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,PROTECH SWAP REPLACE MONTHLY,2020-08-19,VIVO,NaN,2019-11-01,VIVO X50 PRO,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0
368121,202112,Thailand,True,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,PROTECH SWAP REPLACE MONTHLY,2020-08-19,XIAOMI,NaN,2020-01-01,XIAOMI MI 10T,...,0,0,0,0,0,0.0,0.0,0.0,0.0000,0


In [77]:
# Adjustable Settings
predict_level = 'SKU' #['SKU','Capacity','Family']

#### ***** DO NOT MAKE FALSE *****
train_bfc = True #Train models for back forecast validation. Leave as default always True, False is not compatible yet
back_forecast = 3 #Months to predict backwards for validation

train_ffc = False #Train models for forward future forecast
forward_forecast = 3 #Months to predict into the future

country_train = True #Train with country or without country
country_input = ['Singapore','Malaysia','Thailand'] #['Singapore','Malaysia','Thailand']

sku_make_train = ['APPLE','Samsung','OPPO','HUAWEI','VIVO'] #['Apple','Samsung','Oppo','Huawei','VIVO']

lag1 = True #Include lag, previous month
lag2 = False #Include lag2, previous number of months gap
lag2_gap = 2 #If lag2 is True, number of months gap between lag

param_tune = False #True or False, tuning takes a while depending on tune time set
tune_time = 300 #in seconds, for each of the three XGB models, so if 300 seconds (5 minutes) for three models, total time is 15 minutes
metric = 'mae' #['mae','mse','r2']

minimum_claims = 10 #Minimum number of claims needed per month to be considered for model evaluation, 10 = Runner Phones

demand_categorization = False
demand_category = 'Runners and Repeaters' #[Runners, Repeaters, Strangers, Runners and Repeaters]

price_info = True
if price_info == False:
  imported_data['Currency'] = imported_data['Currency'].fillna(0)
  imported_data['Price'] = imported_data['Price'].fillna(0)

#Algorithms Model types to train
regular_xgb = True #Always leave as True
no_pre_xgb = False

#Final Output
show_graphs = True
specific_output = False
output_country = 'Singapore'
output_make = 'Apple'
output_family = 'APPLE IPHONE 14 PRO'
output_capacity = 256
output_colour = 'Purple'
specific_predecessor = False
output_pre_country = 'Singapore'
output_pre_make = 'Apple'
output_pre_family = 'APPLE IPHONE 11 PRO'
output_pre_capacity = 256
output_pre_colour = 'Grey'

In [78]:
inventory_data = imported_data.copy()

if country_input != []:
  inventory_data = inventory_data[inventory_data['Country'].isin(country_input)]

if sku_make_train != []:
  inventory_data = inventory_data[inventory_data['Make'].isin(sku_make_train)]

number = inventory_data

inventory_data = inventory_data[['YearMonth','Country','Make','Model_Family','SKU','SKU_No_Color','phone_size',
                                 'Closing_base', 'Shipped_Claim','currency','Price_USD','Client','program','product','product_launch_date']]
inventory_data = inventory_data.rename(columns={
    'Model_Family': 'Model Family',
    'Price_USD':'Price',
    'SKU_No_Color':'SKU No Colour',
    'Closing_base':'Closing Subs Monthly',
    'program':'Program',
    'product':'Product',
    'product_launch_date':'Product launch date',
    'phone_size':'Size',
    'Shipped_Claim':'Claims',
    'currency':'Currency'

})

#Only keep models that have at least 1 claim throughout its lifetime, no point training models with 0 claims
grouped = inventory_data.groupby('SKU')['Claims'].sum().reset_index()
filtered = grouped[grouped['Claims'] > 0]
inventory_data = inventory_data[inventory_data['SKU'].isin(filtered['SKU'])]

#Convert YearMonth column to datetime
inventory_data['YearMonth'] = inventory_data['YearMonth'].astype(str)
inventory_data['YearMonth'] = pd.to_datetime(inventory_data['YearMonth'], format='%Y%m')
inventory_data['YearMonth'] = inventory_data['YearMonth'].dt.to_period('M')

#Map Missing Phone Series to respective Model Families
'''mapping_dict = {
    'OPPO A16': 'OPPO A10',
    'OPPO A5 2020': 'OPPO A50',
    'OPPO A94': 'OPPO A90',
    'SAMSUNG GALAXY A10S': 'SAMSUNG GALAXY A10',
    'SAMSUNG GALAXY A10': 'SAMSUNG GALAXY A10',
    'SAMSUNG GALAXY J4 PLUS 2018': 'SAMSUNG GALAXY J4',
    'SAMSUNG GALAXY NOTE 9': 'SAMSUNG GALAXY NOTE 9'
}
inventory_data['Phone Series'] = inventory_data['Model Family'].map(mapping_dict).fillna(inventory_data['Model Family'])'''
#inventory_data['Phone Series'] = inventory_data['Phone Series'].fillna('Phone Series')
inventory_data['Claims'] = inventory_data['Claims'].fillna(0)

In [79]:
inventory_data

,YearMonth,Country,Make,Model Family,SKU,SKU No Colour,Size,Closing Subs Monthly,Claims,Currency,Price,Client,Program,Product,Product launch date
0,2019-01,Malaysia,Samsung,SAMSUNG GALAXY A02S,GXYA02S64BLK,GXYA02S64,64,42,0,NaN,127.469883,Celcom,Celcom - Swap - Monthly,Celcom_SwapOnly_Monthly,2015-11-24
1,2019-01,Malaysia,Samsung,SAMSUNG GALAXY A02S,GXYA02S64BLU,GXYA02S64,64,35,0,NaN,127.469883,Celcom,Celcom - Swap - Monthly,Celcom_SwapOnly_Monthly,2015-11-24
2,2019-01,Malaysia,Samsung,SAMSUNG GALAXY A03,GXYA03464BLK,GXYA03464,64,26,0,NaN,132.289160,Celcom,Celcom - Swap - Monthly,Celcom_SwapOnly_Monthly,2015-11-24
3,2019-01,Malaysia,Samsung,SAMSUNG GALAXY A03,GXYA03464BLU,GXYA03464,64,33,0,NaN,132.289160,Celcom,Celcom - Swap - Monthly,Celcom_SwapOnly_Monthly,2015-11-24
4,2019-01,Malaysia,Samsung,SAMSUNG GALAXY A03S,GXYA03S64BLK,GXYA03S64,64,27,0,NaN,134.698798,Celcom,Celcom - Swap - Monthly,Celcom_SwapOnly_Monthly,2015-11-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368107,2021-12,Thailand,APPLE,APPLE IPAD PRO 12.9 2021,IPRO1221512SIL,IPRO1221512,512,1,0,NaN,1722.577211,True,PROTECH SWAP REPLACE MONTHLY,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,2020-08-19
368117,2021-12,Thailand,VIVO,VIVO V20 PRO,V20PRO5G128BLK,V20PRO5G128,128,10,0,NaN,483.837745,True,PROTECH SWAP REPLACE MONTHLY,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,2020-08-19
368118,2021-12,Thailand,VIVO,VIVO V20 PRO,V20PRO5G128PUR,V20PRO5G128,128,37,0,NaN,483.837745,True,PROTECH SWAP REPLACE MONTHLY,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,2020-08-19
368119,2021-12,Thailand,VIVO,VIVO V20 PRO,V20PRO5G128WHT,V20PRO5G128,128,5,0,NaN,483.837745,True,PROTECH SWAP REPLACE MONTHLY,TRUE_Protech_Swap_Replace_Monthly_6+6_Free,2020-08-19


In [80]:
#Adding release and discontinued date, predecessor and succesor model family
release_data = release_data[['Model Family', 'Release Date', 'Discontinued Date','Predecessor','Succesor']]
inventory_data = inventory_data.merge(release_data, left_on='Model Family', right_on='Model Family', how='left')
inventory_data['Release Date'] = pd.to_datetime(inventory_data['Release Date'])
inventory_data['Discontinued Date'] = pd.to_datetime(inventory_data['Discontinued Date'])
inventory_data['Release Date'] = inventory_data['Release Date'].dt.to_period('M').dt.to_timestamp()
inventory_data['Discontinued Date'] = inventory_data['Discontinued Date'].dt.to_period('M').dt.to_timestamp()
inventory_data['Predecessor'] = inventory_data['Predecessor'].fillna('PREVIOUS PHONE SERIES')
inventory_data['Succesor'] = inventory_data['Succesor'].fillna('NEXT PHONE SERIES')

#Adding release and discontinued date of predecessor and succesor
release_data = release_data.drop(['Predecessor','Succesor'],axis=1)
pre_release = release_data.copy().rename(columns={'Model Family': 'Predecessor', 'Release Date': 'Pre_Release'})
suc_release = release_data.copy().rename(columns={'Model Family': 'Succesor', 'Release Date': 'Suc_Release'})
merged_df = inventory_data.merge(pre_release[['Predecessor','Pre_Release']], left_on='Predecessor', right_on='Predecessor',how='left')
merged_df = merged_df.merge(suc_release[['Succesor','Suc_Release']], left_on='Succesor', right_on='Succesor',how='left')
merged_df['Pre_Release'] = pd.to_datetime(merged_df['Pre_Release'])
merged_df['Suc_Release'] = pd.to_datetime(merged_df['Suc_Release'])
merged_df['Pre_Release'] = merged_df['Pre_Release'].fillna(pd.Timestamp('2015-01-01'))
merged_df['Suc_Release'] = merged_df['Suc_Release'].fillna(pd.Timestamp('2024-01-01'))
merged_df['Product launch date'] = merged_df['Product launch date'].fillna(pd.Timestamp('2024-01-01'))

#Get moodel age column and remove data where model age is less than 0
merged_df['YearMonth'] = merged_df['YearMonth'].astype(str)
merged_df['YearMonth'] = pd.to_datetime(merged_df['YearMonth'])
merged_df['MONTH'] = merged_df['YearMonth'].dt.month
merged_df['YEAR'] = merged_df['YearMonth'].dt.year
merged_df['RELEASE_MONTH'] = merged_df['Release Date'].dt.month
merged_df['RELEASE_YEAR'] = merged_df['Release Date'].dt.year
merged_df['Model Age'] = ((merged_df['YearMonth'].dt.year - merged_df['Release Date'].dt.year) * 12 +
                          (merged_df['YearMonth'].dt.month - merged_df['Release Date'].dt.month))
merged_df = merged_df[merged_df['Model Age']>0]

#Fix storage size
merged_df['Size'] = merged_df['Size'].replace('1', '1000')
merged_df['Size'] = merged_df['Size'].replace('None', '0')
merged_df['Size'] = merged_df['Size'].fillna('0')
merged_df['Size'] = merged_df['Size'].str.replace('GB', '')
merged_df['Storage'] = merged_df['Size'].astype(int)
merged_df = merged_df.drop('Size',axis=1)

#Make new unique Country_level
if predict_level == 'SKU':
  merged_df['Country_Level'] = merged_df['Client'].astype(str) + merged_df['Product'] + merged_df['Country'] + merged_df['SKU']
elif predict_level == 'Capacity':
  merged_df['Country_Level'] = merged_df['Client'].astype(str) + merged_df['Product'] + merged_df['Country'] + merged_df['SKU No Colour']
elif predict_level == 'Family':
  merged_df['Country_Level'] = merged_df['Client'].astype(str) + merged_df['Product'] + merged_df['Country'] + merged_df['Model Family']

#Remove final month from data (usually month's data is incomplete i.e. Today is 25 Nov 2023, so the full month Nov data is incomplete)
merged_df = merged_df[merged_df['YearMonth']!=merged_df['YearMonth'].max()]

#Additional Evaluation Stuff
'''if back_forecast == 2:
  merged_df = merged_df[merged_df['YearMonth']!=merged_df['YearMonth'].max()]
elif back_forecast == 1:
  merged_df = merged_df[merged_df['YearMonth']!=merged_df['YearMonth'].max()]
  merged_df = merged_df[merged_df['YearMonth']!=merged_df['YearMonth'].max()]'''

#Price fixing
price_df = merged_df[['Country_Level','Price','Model Age']].drop_duplicates().copy()
price_df = price_df[price_df['Price']>100]
price_df = price_df.groupby('Country_Level')['Price'].mean().reset_index()
currency = {'Currency': ['SGD', 'MYR', 'THB'],'Country': ['Singapore', 'Malaysia', 'Thailand']}
currency_df = pd.DataFrame(currency)

merged_df = merged_df.drop(['Price','Currency'],axis=1)
merged_df = merged_df.merge(price_df, left_on='Country_Level',right_on='Country_Level',how='left')
merged_df = merged_df.merge(currency_df, left_on='Country',right_on='Country',how='left')

'''merged_df.loc[merged_df['Country_Level']=='ThailandA3S32GSLRED', 'Price'] = 4086.99
merged_df.loc[merged_df['Country_Level']=='ThailandY7PRO19SLBLK', 'Price'] = 3252.33
merged_df.loc[merged_df['Country_Level']=='SingaporeIPH1164PUR', 'Price'] = 1081.960000
merged_df.loc[merged_df['Country_Level']=='MalaysiaIPH1164PUR', 'Price'] = 3091.328182
merged_df.loc[merged_df['Country_Level']=='MalaysiaA1664SIL', 'Price'] = 590.145652'''

/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/3288453153.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  inventory_data['Release Date'] = pd.to_datetime(inventory_data['Release Date'])
/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/3288453153.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  inventory_data['Discontinued Date'] = pd.to_datetime(inventory_data['Discontinued Date'])
/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/3288453153.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['Pre_Release'] 

"merged_df.loc[merged_df['Country_Level']=='ThailandA3S32GSLRED', 'Price'] = 4086.99\nmerged_df.loc[merged_df['Country_Level']=='ThailandY7PRO19SLBLK', 'Price'] = 3252.33\nmerged_df.loc[merged_df['Country_Level']=='SingaporeIPH1164PUR', 'Price'] = 1081.960000\nmerged_df.loc[merged_df['Country_Level']=='MalaysiaIPH1164PUR', 'Price'] = 3091.328182\nmerged_df.loc[merged_df['Country_Level']=='MalaysiaA1664SIL', 'Price'] = 590.145652"

In [81]:
df = merged_df.copy()

#More preprocessing related to predecessor and succesor
df['STATUS'] = 'BAU'  # Default status

for index, row in df.iterrows():
    if row['YearMonth'] < row['Release Date']:
        df.at[index, 'STATUS'] = 'UNRELEASED'
    elif row['YearMonth'] > row['Discontinued Date']:
        df.at[index, 'STATUS'] = 'EOL'

#Succesor Release
df['Succesor Status'] = 0  # Default status, unreleased
df.loc[df['YearMonth'] >= df['Suc_Release'], 'Succesor Status'] = 1

df['Colour'] = df['SKU'].str[-3:]

if predict_level == 'SKU':
  base_pre_list = ['Model Age','Claims','Country_Level','SKU','Storage','Model Family']
  pre_de_list = ['Country_Level','Storage','Model Family','Country','Client','Product','Program','Colour']
  merge_list = ['Storage','Predecessor','Country','Client','Product','Program','Colour']
elif predict_level == 'Capacity':
  base_pre_list = ['Model Age','Claims','Country_Level','SKU No Colour','Storage','Model Family']
  pre_de_list = ['Country_Level','Storage','Model Family','Country','Client','Product','Program']
  merge_list = ['Storage','Predecessor','Country','Client','Product','Program']
else: #Model Family
  base_pre_list = ['Model Age','Claims','Country_Level','Storage','Model Family']
  pre_de_list = ['Country_Level','Model Family','Country','Client','Product','Program']
  merge_list = ['Predecessor','Country','Client','Product','Program']

#Merge claims based on Predecessor, Model Age (Months), Storage
pre_df = df[base_pre_list].copy().rename(columns={'Country_Level': 'Predecessor', 'Claims': 'Pre Claims'}).drop_duplicates()
pre_details = df[pre_de_list].copy().drop_duplicates()
pre_details = pre_details.rename(columns={'Country_Level': 'unique_id2','Model Family':'Predecessor'})

df = df.merge(pre_details, left_on=merge_list,right_on=merge_list,how='left')
df = df.drop('Predecessor',axis=1)
df = df.rename(columns={'unique_id2': 'Predecessor'})
df = df.merge(pre_df[['Model Age','Pre Claims','Predecessor']], left_on=['Predecessor','Model Age'], right_on=['Predecessor','Model Age'],how='left')
df = df.drop(['Release Date','Discontinued Date','Suc_Release','Pre_Release'],axis = 1)
df['Predecessor'] = df['Predecessor'].fillna('PREVIOUS PHONE SERIES')

df.loc[df['STATUS'] == 'UNRELEASED', 'Pre Claims'] = 0
df = df.fillna(0)

In [82]:
#Remove data after gaps in monthly data (if a model has a gap in their data, it can be assumed that they are being claimed too infrequently)
df.sort_values(['Country_Level', 'Model Age'], inplace=True)
df = df.reset_index(drop=True)
def remove_after_gap(group):
    diff = group['Model Age'].diff()
    if any(diff > 1):
        idx = diff[diff > 1].index[0]
        return group.loc[:idx-1]
    else:
        return group
df = df.groupby('Country_Level').apply(remove_after_gap).reset_index(drop=True)

/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/1265977495.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Country_Level').apply(remove_after_gap).reset_index(drop=True)


In [83]:
# Make Model Type columns
type_patterns = ['PLUS', 'ULTRA', 'FOLD', 'EDGE', 'DUO', 'FLIP', 'FE','STAR','LITE','PRO MAX', 'PRO', 'MINI', 'MAX',]

# Extract MODEL_TYPE based on patterns
df['Model Type'] = df['Model Family'].str.extract('(' + '|'.join(type_patterns) + ')', expand=False)

# Fill missing values in MODEL_TYPE with 'Regular'
df['Model Type'].fillna('BASIC', inplace=True)

df = df.rename(columns={
    'Country_Level': 'unique_id',
    'YearMonth': 'ds',
    'Claims': 'y',
})

if predict_level == 'SKU':
  groupby_list = ['unique_id', 'ds', 'Model Age','Country','Make','Predecessor','Succesor','STATUS','MONTH','YEAR','RELEASE_MONTH','RELEASE_YEAR',
                 'Succesor Status','Model Type','Storage','Model Family','SKU','Currency','Price','SKU No Colour',
                  'Client','Program','Product','Product launch date']
elif predict_level == 'Capacity':
  groupby_list = ['unique_id', 'ds', 'Model Age','Country','Make','Predecessor','Succesor','STATUS','MONTH','YEAR','RELEASE_MONTH','RELEASE_YEAR',
                 'Succesor Status','Model Type','Storage','Model Family','SKU No Colour','Currency','Price',
                  'Client','Program','Product','Product launch date']
else: #Model Family
  groupby_list = ['unique_id', 'ds', 'Model Age','Country','Make','Predecessor','Succesor','STATUS','MONTH','YEAR','RELEASE_MONTH','RELEASE_YEAR',
                 'Succesor Status','Model Type','Model Family','Currency','Price',
                  'Client','Program','Product','Product launch date']

df = df.groupby(groupby_list)[['y','Pre Claims','Closing Subs Monthly']].sum().reset_index()
# df = df.groupby(groupby_list)['y','Pre Claims','Closing Subs Monthly'].sum().reset_index() use either one of these if Cannot subset "columns with a tuple with more than one element. Use a list instead." occurs

/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/1377351794.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Model Type'].fillna('BASIC', inplace=True)


In [84]:
df.sort_values(by='ds', inplace=True)
'''df['Pre Claims Total'] = df.groupby('unique_id')['Pre Claims'].rolling(6).sum().reset_index(0,drop=True)
df['Pre 3 Claims'] = df.groupby('unique_id')['y'].rolling(6).sum().reset_index(0,drop=True)
df['Pre Difference'] = df.groupby('unique_id').apply(lambda group: (group['Pre 3 Claims'] - group['Pre Claims Total']) / 3).reset_index(0,drop=True)
df = df.drop(['Pre Claims Total','Pre 3 Claims'],axis=1)'''
df.fillna(0, inplace=True)

if lag1:
  y = df['y'].copy()
  df['Lag'] = df.groupby(['unique_id','Country','Client','Product'])['y'].shift(1)
if lag2:
  y = df['y'].copy()
  df['Lag2'] = df.groupby(['unique_id','Country','Client','Product'])['y'].shift(lag2_gap)
df.fillna(0, inplace=True)

In [85]:
df[(df['Country']=='Singapore')&(df['SKU']=='IPH13PRO5G256GRY')]['unique_id'].unique()

array(['StarHubSmartSupport Basic 3SingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop Unlimited PremiumSingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop_Unlimited_24MSingaporeIPH13PRO5G256GRY',
       'M1M1-FoneCare+ ProSingaporeIPH13PRO5G256GRY',
       'StarHubSmartSupport LiteSingaporeIPH13PRO5G256GRY',
       'M1M1_SwapReplace_Monthly_RefreshSingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop Unlimited Premium (24 mths term)SingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop Unlimited (12 months)SingaporeIPH13PRO5G256GRY',
       'SingTelEnterprise Device Care (24 mths)SingaporeIPH13PRO5G256GRY',
       'StarHubSmartSupport ($12)SingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop UnlimitedSingaporeIPH13PRO5G256GRY',
       'StarHubBiz+ SmartSupportSingaporeIPH13PRO5G256GRY',
       'SingTelEnterprise Device Care Lite - 24 monthsSingaporeIPH13PRO5G256GRY',
       'StarHubSmartSupport Basic $10SingaporeIPH13PRO5G256GRY',
       'SingTelMobileSwop Unlimited Premium (12 mt

In [86]:
df.tail()

,unique_id,ds,Model Age,Country,Make,Predecessor,Succesor,STATUS,MONTH,YEAR,...,Price,SKU No Colour,Client,Program,Product,Product launch date,y,Pre Claims,Closing Subs Monthly,Lag
10967,AISAIS_SinglePricePoint_Swap_Replace_FixedThai...,2023-11-01,21.0,Thailand,Samsung,PREVIOUS PHONE SERIES,SAMSUNG GALAXY S23,EOL,11,2023,...,937.501999,GXYS225G256,AIS,MobileCare SPP Swap & Replace 12 month,AIS_SinglePricePoint_Swap_Replace_Fixed,2014-07-02,0,0.0,0,0.0
674193,TrueTRUE_Protech_Swap_Replace_Monthly_3+3Thail...,2023-11-01,50.0,Thailand,APPLE,PREVIOUS PHONE SERIES,APPLE IPHONE 12 PRO MAX,EOL,11,2023,...,1228.624738,IPH11PRM64,True,Protech Swap Replace Monthly,TRUE_Protech_Swap_Replace_Monthly_3+3,2019-11-21,0,0.0,0,0.0
471315,SingTelMobileSwop UnlimitedSingaporeGXYS9BLK,2023-11-01,68.0,Singapore,Samsung,SingTelMobileSwop UnlimitedSingaporeGXYS8BLK,SAMSUNG GALAXY S10,EOL,11,2023,...,894.349567,GXYS9,SingTel,MobileSwop Unlimited,MobileSwop Unlimited,2014-09-03,0,0.0,47,0.0
196711,AISAIS_SwapReplace_12mthThailandIPH7P32JET,2023-11-01,86.0,Thailand,APPLE,PREVIOUS PHONE SERIES,APPLE IPHONE 8 PLUS,EOL,11,2023,...,848.839412,IPH7P32,AIS,MobileCare Swap & Replace 12 month,AIS_SwapReplace_12mth,2014-07-02,0,0.0,0,0.0
702810,TrueTrue_Protech_Plus_Monthly_6+F6_FreeThailan...,2023-11-01,37.0,Thailand,APPLE,PREVIOUS PHONE SERIES,APPLE IPHONE 13 PRO,EOL,11,2023,...,1049.804970,IPH12PRO5G128,True,True_Protech_Plus_Monthly,True_Protech_Plus_Monthly_6+F6_Free,2023-06-07,0,0.0,0,0.0


In [87]:
#Forward Forecast Data
def get_next_n_months(start_date_str, n):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    return [(start_date + relativedelta(months=i)).strftime('%Y-%m-%d') for i in range(1, n + 1)]

if train_ffc:
  current_month_df = df[df['ds']==df['ds'].max()]

  combinations = list(product(current_month_df['unique_id'].unique().tolist(), get_next_n_months(current_month_df['ds'].astype(str).unique()[0],forward_forecast)))
  combi_df = pd.DataFrame(combinations, columns=['unique_id', 'ds'])
  combi_df['ds'] = pd.to_datetime(combi_df['ds'])

  current_month_df = current_month_df.drop(['ds','Model Age','MONTH','YEAR'],axis=1)
  combi_df = combi_df.merge(current_month_df,left_on='unique_id',right_on=['unique_id'],how='left')
  combi_df = combi_df.merge(release_data[['Model Family','Release Date']],left_on='Model Family',right_on='Model Family',how='left')

  combi_df['ds'] = pd.to_datetime(combi_df['ds'])
  combi_df['Release Date'] = pd.to_datetime(combi_df['Release Date'])
  combi_df['MONTH'] = combi_df['ds'].dt.month
  combi_df['YEAR'] = combi_df['ds'].dt.year
  combi_df['Model Age'] = ((combi_df['ds'].dt.year - combi_df['Release Date'].dt.year) * 12 +
                            (combi_df['ds'].dt.month - combi_df['Release Date'].dt.month))
  combi_df = combi_df.drop('Release Date',axis=1)

  combi_df = combi_df.drop('Pre Claims',axis=1)
  base_pre_list = ['Model Age','y','unique_id','SKU','Storage','Model Family','Colour']
  pre_de_list = ['unique_id','Storage','Model Family','Colour','Country','Client','Product']
  merge_list = ['Storage','Predecessor','Colour','Country','Client','Product']
  pre_df = df[base_pre_list].copy().rename(columns={'unique_id': 'Predecessor', 'y': 'Pre Claims'}).drop_duplicates()
  combi_df = combi_df.merge(pre_df[['Model Age','Pre Claims','Predecessor']], left_on=['Predecessor','Model Age'], right_on=['Predecessor','Model Age'],how='left')

  final_df = pd.concat([df, combi_df], ignore_index=True)

In [88]:
if train_ffc != True:
  final_df = df.copy()

# Perform label encoding on 'MODEL_FAMILY' and 'unique_id' columns
label_encoder = LabelEncoder()

#final_df['Phone Series'] = label_encoder.fit_transform(final_df['Phone Series'])
final_df['Predecessor'] = label_encoder.fit_transform(final_df['Predecessor'])
final_df['Succesor'] = label_encoder.fit_transform(final_df['Succesor'])
final_df['STATUS'] = label_encoder.fit_transform(final_df['STATUS'])
final_df['Model Type'] = label_encoder.fit_transform(final_df['Model Type'])
final_df['Make'] = label_encoder.fit_transform(final_df['Make'])
final_df['Client'] = label_encoder.fit_transform(final_df['Client'])
final_df['Program'] = label_encoder.fit_transform(final_df['Program'])
final_df['Product'] = label_encoder.fit_transform(final_df['Product'])
final_df['Product launch date'] = label_encoder.fit_transform(final_df['Product launch date'])

if predict_level == 'SKU':
  final_df['Model Family'] = label_encoder.fit_transform(final_df['Model Family'])
  #final_df['Colour'] = label_encoder.fit_transform(final_df['Colour'])
  final_df['SKU'] = label_encoder.fit_transform(final_df['SKU'])
  final_df['SKU No Colour'] = label_encoder.fit_transform(final_df['SKU No Colour'])
elif predict_level == 'Capacity':
  final_df['Model Family'] = label_encoder.fit_transform(final_df['Model Family'])
  final_df['SKU No Colour'] = label_encoder.fit_transform(final_df['SKU No Colour'])
else:
  final_df['Model Family'] = label_encoder.fit_transform(final_df['Model Family'])

if country_train:
  clabel_encoder = LabelEncoder()
  final_df['Country'] = clabel_encoder.fit_transform(final_df['Country'])
  final_df['Currency'] = clabel_encoder.fit_transform(final_df['Currency'])

final_df['unique_id'] = label_encoder.fit_transform(final_df['unique_id'])

# Perform ordered ordinal encoding on 'ts' column
ordinal_encoder = OrdinalEncoder()

# Reshape 'ds' to a 2D array as OrdinalEncoder expects a 2D array
ts_reshaped = final_df['ds'].values.reshape(-1, 1)
final_df['ds'] = ordinal_encoder.fit_transform(ts_reshaped)
final_df['ds'] = final_df['ds'].astype(int)
final_df = final_df.drop(['Currency'],axis=1)
if price_info == False:
  final_df = final_df.drop(['Price'],axis=1)

In [89]:
final_df

,unique_id,ds,Model Age,Country,Make,Predecessor,Succesor,STATUS,MONTH,YEAR,...,Price,SKU No Colour,Client,Program,Product,Product launch date,y,Pre Claims,Closing Subs Monthly,Lag
301825,7474,0,16.0,0,0,1868,38,0,1,2019,...,692.301750,448,4,7,31,9,0,0.0,56,0.0
297431,7391,0,3.0,0,0,1868,16,0,1,2019,...,632.541429,297,4,7,31,9,0,0.0,41,0.0
324278,8220,0,19.0,1,0,1868,10,0,1,2019,...,871.953501,200,5,19,48,14,0,0.0,22,0.0
573659,15533,0,10.0,1,3,3336,71,0,1,2019,...,894.349567,139,8,50,75,20,0,0.0,10,0.0
214551,4440,0,4.0,2,0,1868,18,0,1,2019,...,1355.056370,301,0,29,14,4,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10967,218,58,21.0,2,3,1868,84,1,11,2023,...,937.501999,120,0,27,0,4,0,0.0,0,0.0
674193,18313,58,50.0,2,0,1868,21,1,11,2023,...,1228.624738,213,9,46,98,25,0,0.0,0,0.0
471315,12627,58,68.0,1,3,2615,71,1,11,2023,...,894.349567,139,7,36,59,7,0,0.0,47,0.0
196711,4132,58,86.0,2,0,1868,35,1,11,2023,...,848.839412,275,0,29,12,4,0,0.0,0,0.0


In [90]:
# Extract dates for train and test set
total_months = df['ds'].nunique()
X = final_df.copy()

if train_ffc:
  dates = X['ds'].unique()
  dtrain = dates[:-forward_forecast]
  dtest = dates[-forward_forecast:]
  fortrain_df = X.query('ds in @dtrain')
  forward_df = X.query('ds in @dtest')
  X = fortrain_df.copy()

forecast = back_forecast
dates = X['ds'].unique()
dtrain = dates[:-forecast]
dtest = dates[-forecast:]

train = X.query('ds in @dtrain')
test = X.query('ds in @dtest')

def train_models(train,test,actualpredicted,current_model):
  if param_tune != True:
    model = xgb.XGBRegressor(n_estimators=586,
                            max_leaves=154,
                            min_child_weight=1.498295437083999,
                            learning_rate=0.043253199923420985,
                            subsample=0.8061837317376596,
                            colsample_bylevel=1.0,
                            colsample_bytree=1.0,
                            reg_alpha=0.28935196499159016,
                            reg_lambda=85.25098834625248)
    model.fit(train.drop('y',axis=1), train[['y']])
    preds = model.predict(test.drop('y',axis=1))
    '''else:
    automl = AutoML()
    settings = {
        "time_budget": tune_time,  # total running time in seconds
        "metric": metric,  # primary metrics for regression can be chosen from: ['mae','mse','r2']
        "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
        "task": 'regression',  # task type
        "seed": 42,  # random seed
    }
    automl.fit(X_train=train.drop('y',axis=1), y_train=train['y'], **settings)
    preds = automl.predict(test.drop('y',axis=1))
    print('Best hyperparmeter config:', automl.best_config)
    print('Best r2 on validation data: {0:.4g}'.format(1-automl.best_loss))
    print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))
    print(automl.model.estimator)'''

  if current_model == 'regular xgb':
    actualpredicted['XGB'] = preds
  if current_model == 'no pred xgb':
    actualpredicted['XGB No Predecessor Claims'] = preds

  return actualpredicted

if regular_xgb:
  current_model = 'regular xgb'
  actualpredicted = test[['y','unique_id','ds']].copy()

  if param_tune != True:
    unique_ds_counts = actualpredicted.groupby('unique_id')['ds'].nunique()
    valid_unique_ids = unique_ds_counts[unique_ds_counts == back_forecast].index
    actualpredicted = actualpredicted[actualpredicted['unique_id'].isin(valid_unique_ids)]
    test = test[test['unique_id'].isin(valid_unique_ids)]

  actualpredicted = train_models(train,test,actualpredicted,current_model)
  if train_ffc:
    forward_prediction = forward_df[['y','unique_id','ds']].copy()
    forward_prediction = train_models(fortrain_df,forward_df,forward_prediction,current_model)
if no_pre_xgb:
  current_model = 'no pred xgb'
  actualpredicted = train_models(train.drop(['Predecessor','Pre Claims'],axis=1),test.drop(['Predecessor','Pre Claims'],axis=1),actualpredicted,current_model)
  if train_ffc:
    forward_prediction = train_models(fortrain_df.drop(['Predecessor','Pre Claims'],axis=1),forward_df.drop(['Predecessor','Pre Claims'],axis=1),forward_prediction,current_model)

In [91]:
train

,unique_id,ds,Model Age,Country,Make,Predecessor,Succesor,STATUS,MONTH,YEAR,...,Price,SKU No Colour,Client,Program,Product,Product launch date,y,Pre Claims,Closing Subs Monthly,Lag
301825,7474,0,16.0,0,0,1868,38,0,1,2019,...,692.301750,448,4,7,31,9,0,0.0,56,0.0
297431,7391,0,3.0,0,0,1868,16,0,1,2019,...,632.541429,297,4,7,31,9,0,0.0,41,0.0
324278,8220,0,19.0,1,0,1868,10,0,1,2019,...,871.953501,200,5,19,48,14,0,0.0,22,0.0
573659,15533,0,10.0,1,3,3336,71,0,1,2019,...,894.349567,139,8,50,75,20,0,0.0,10,0.0
214551,4440,0,4.0,2,0,1868,18,0,1,2019,...,1355.056370,301,0,29,14,4,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650822,17459,55,19.0,2,3,1868,82,1,8,2023,...,676.381263,112,9,45,96,40,0,0.0,1,0.0
354728,8937,55,23.0,1,0,1723,26,0,8,2023,...,1092.977606,232,5,17,49,14,0,0.0,50,0.0
396288,10118,55,23.0,1,0,2036,27,0,8,2023,...,977.037398,235,7,14,37,8,0,0.0,1,0.0
116458,2623,55,74.0,2,0,1868,10,1,8,2023,...,910.573187,200,0,28,4,4,0,0.0,0,0.0


In [92]:
test

,unique_id,ds,Model Age,Country,Make,Predecessor,Succesor,STATUS,MONTH,YEAR,...,Price,SKU No Colour,Client,Program,Product,Product launch date,y,Pre Claims,Closing Subs Monthly,Lag
8185,150,56,59.0,2,3,1868,56,1,9,2023,...,144.765703,87,0,27,0,4,0,0.0,0,0.0
673553,18297,56,48.0,2,0,1868,19,1,9,2023,...,828.320939,206,9,46,98,25,0,0.0,0,0.0
660870,17869,56,13.0,2,3,1868,49,0,9,2023,...,1888.111837,166,9,46,97,24,0,0.0,5,0.0
651965,17509,56,13.0,2,3,3654,102,0,9,2023,...,1044.465008,154,9,45,96,40,0,0.0,3,0.0
665471,18008,56,24.0,2,0,1868,26,0,9,2023,...,1213.729248,233,9,46,97,24,0,0.0,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10967,218,58,21.0,2,3,1868,84,1,11,2023,...,937.501999,120,0,27,0,4,0,0.0,0,0.0
674193,18313,58,50.0,2,0,1868,21,1,11,2023,...,1228.624738,213,9,46,98,25,0,0.0,0,0.0
471315,12627,58,68.0,1,3,2615,71,1,11,2023,...,894.349567,139,7,36,59,7,0,0.0,47,0.0
196711,4132,58,86.0,2,0,1868,35,1,11,2023,...,848.839412,275,0,29,12,4,0,0.0,0,0.0


In [93]:
# Moving Average Calculation
window_size = forecast
madf = train[['unique_id','ds','y']].copy()
madf['moving_avg'] = madf.groupby('unique_id')['y'].rolling(window_size).mean().reset_index(0, drop=True)
madf['moving_avg'] = madf.groupby('unique_id')['moving_avg'].shift()
if train_ffc:
  next_months = list(range(total_months-forecast-forward_forecast, total_months-forward_forecast))
else:
  next_months = list(range(total_months-forecast, total_months))
unique_ids = madf['unique_id'].unique()
future_df = pd.DataFrame()
for id in unique_ids:
    last_values = madf[madf['unique_id'] == id]['y'].tail(window_size).values
    for month in next_months:
        prediction = np.mean(last_values)
        new_row = pd.DataFrame({  'unique_id': [id],  'ds': [month],  'moving_avg': [prediction]  })
        future_df = pd.concat([future_df, new_row], ignore_index=True)
        last_values = np.append(last_values[1:], prediction)
future_df['unique_id'] = future_df['unique_id'].astype(int)
future_df['ds'] = future_df['ds'].astype(int)
future_df = actualpredicted.merge(future_df, left_on=['unique_id','ds'], right_on=['unique_id','ds'], how='left')

if demand_categorization:
  if train_ffc:
    final_date = final_df['ds'].max()-forward_forecast
  else:
    final_date = final_df['ds'].max()
  demand_skus = final_df[final_df['ds']==final_date][['unique_id','SKU No Colour','y']].copy()
  if demand_category == 'Runners':
    demand_skus = demand_skus[(demand_skus['y']>10)]
  elif demand_category == 'Repeaters':
    demand_skus = demand_skus[(demand_skus['y']<10) & (demand_skus['y']>0)]
  elif demand_category == 'Strangers':
    demand_skus = demand_skus[(demand_skus['y']<1)]
  elif demand_category == 'Runners and Repeaters':
    demand_skus = demand_skus[(demand_skus['y']>0)]
  demand_skus = demand_skus['unique_id'].unique().tolist()
  future_df = future_df[future_df['unique_id'].isin(demand_skus)]
else:
  futue_df = future_df
  #future_df = future_df.groupby('unique_id').filter(lambda x: x['y'].sum() >= forecast*minimum_claims)

if train_ffc:
  window_size = forward_forecast
  madf = fortrain_df[['unique_id','ds','y']].copy()
  madf['moving_avg'] = madf.groupby('unique_id')['y'].rolling(window_size).mean().reset_index(0, drop=True)
  madf['moving_avg'] = madf.groupby('unique_id')['moving_avg'].shift()
  next_months = list(range(total_months-forward_forecast, total_months))
  future2_df = pd.DataFrame()
  for id in future_df['unique_id'].unique():
      last_values = madf[madf['unique_id'] == id]['y'].tail(window_size).values
      for month in next_months:
          prediction = np.mean(last_values)
          new_row = pd.DataFrame({  'unique_id': [id],  'ds': [month],  'moving_avg': [prediction]  })
          future2_df = pd.concat([future2_df, new_row], ignore_index=True)
          last_values = np.append(last_values[1:], prediction)
  future2_df['unique_id'] = future2_df['unique_id'].astype(int)
  future2_df['ds'] = future2_df['ds'].astype(int)
  future2_df = future2_df.merge(forward_prediction,left_on=['unique_id','ds'],right_on=['unique_id','ds'],how='inner')
  future2_df['y'] = np.nan
  future_df = pd.concat([future_df,future2_df],ignore_index=True)
#future_df['unique_id'] = label_encoder.inverse_transform(future_df['unique_id'])
#future_df = future_df[future_df['unique_id'].str.contains('IPH')]

In [94]:
future_df = actualpredicted

In [95]:
future_df['unique_id'].nunique()

19065

In [96]:
plot_df = future_df.copy()
plot_df['unique_id'] = label_encoder.inverse_transform(plot_df['unique_id'])
ts_reshaped = plot_df['ds'].values.reshape(-1, 1)
plot_df['ds'] = ordinal_encoder.inverse_transform(ts_reshaped)

past_df = df[['unique_id','ds','y']].copy()
past_df = past_df[past_df['unique_id'].isin(plot_df['unique_id'].unique().tolist())]
past_df = past_df[past_df['ds'] < plot_df['ds'].min()]

plot_df = pd.concat([plot_df,past_df],ignore_index=True)
plot_df = plot_df.sort_values(by=['unique_id', 'ds'])
months_release = df[['unique_id','Predecessor','Country','Make','Model Family','Storage','Client']].copy().drop_duplicates()
plot_df = plot_df.merge(months_release, left_on=['unique_id'], right_on=['unique_id'], how='left')
months_release = df[['unique_id','ds','Model Age']].copy().drop_duplicates()
plot_df = plot_df.merge(months_release, left_on=['unique_id','ds'], right_on=['unique_id','ds'], how='left')
predecessor_df  = df[['Model Age','unique_id','y']].rename(columns={'unique_id': 'Predecessor', 'y': 'Pre Claims'})
plot_df = plot_df.merge(predecessor_df, left_on=['Model Age','Predecessor'], right_on=['Model Age','Predecessor'], how='left')
plot_df = plot_df[plot_df['Model Age'] > 0]

In [97]:
if specific_output:
  plot_df = plot_df[plot_df['Country']==output_country]
  plot_df = plot_df[plot_df['Make']==output_make]
  plot_df = plot_df[plot_df['Model Family']==output_family]
  plot_df = plot_df[plot_df['Storage']==output_capacity]
  plot_df = plot_df[plot_df['Colour']==output_colour]

  if specific_predecessor:
    plot_df = plot_df.drop(['Predecessor','Pre Claims'],axis=1)
    predecessor_df = df[['unique_id','Model Age','y']].copy().drop_duplicates().rename(columns={'unique_id': 'Predecessor', 'y': 'Pre Claims'})
    predecessor_df = predecessor_df[predecessor_df['Predecessor']==output_predecessor]
    plot_df = plot_df.merge(predecessor_df, left_on=['Model Age'], right_on=['Model Age'], how='left')

In [98]:
#future_df['XGB'] = np.floor(future_df['XGB'])
#future_df['XGB No Predecessor Claims'] = np.floor(future_df['XGB No Predecessor Claims'])
#future_df['moving_avg'] = np.floor(future_df['moving_avg'])

In [99]:
future_df

,y,unique_id,ds,XGB
8185,0,150,56,0.035809
673553,0,18297,56,-0.018241
660870,0,17869,56,0.013751
651965,0,17509,56,0.024767
665471,0,18008,56,-0.008316
...,...,...,...,...
10967,0,218,58,0.008130
674193,0,18313,58,0.000828
471315,0,12627,58,0.077552
196711,0,4132,58,-0.004924


In [100]:
train.columns.tolist()

['unique_id',
 'ds',
 'Model Age',
 'Country',
 'Make',
 'Predecessor',
 'Succesor',
 'STATUS',
 'MONTH',
 'YEAR',
 'RELEASE_MONTH',
 'RELEASE_YEAR',
 'Succesor Status',
 'Model Type',
 'Storage',
 'Model Family',
 'SKU',
 'Price',
 'SKU No Colour',
 'Client',
 'Program',
 'Product',
 'Product launch date',
 'y',
 'Pre Claims',
 'Closing Subs Monthly',
 'Lag']

In [101]:
copy = actualpredicted.copy()
copy['unique_id'] = label_encoder.inverse_transform(copy['unique_id'])
ts_reshaped = copy['ds'].values.reshape(-1, 1)
copy['ds'] = ordinal_encoder.inverse_transform(ts_reshaped)
info = df[['unique_id','Country','SKU','y','Product','Program']].copy().drop_duplicates()
info = info.drop('y',axis=1)
copy = copy.merge(info, left_on=['unique_id'], right_on=['unique_id'], how='left')
copy['unique_id'] = copy['Country'] + copy['SKU']

In [102]:
copy = actualpredicted.copy()
copy['unique_id'] = label_encoder.inverse_transform(copy['unique_id'])
ts_reshaped = copy['ds'].values.reshape(-1, 1)
copy['ds'] = ordinal_encoder.inverse_transform(ts_reshaped)
info = df[['unique_id','Country','SKU','Product','Program','ds']].copy().drop_duplicates()
copy = copy.merge(info, left_on=['unique_id','ds'], right_on=['unique_id','ds'], how='left')
copy['unique_id'] = copy['Country'] + copy['SKU']
#copy = copy.groupby(['unique_id','ds','Country','SKU'])['XGB','y'].sum().reset_index().drop_duplicates()
copy = copy.groupby(['unique_id','ds','Country','SKU'])[['XGB','y']].sum().reset_index().drop_duplicates()

In [103]:
copy[(copy['unique_id'].isin(["SingaporeIPH13PRO5G256GRY","ThailandIPH13PRM5G256BLU","SingaporeIPH14PRM5G256BLK","SingaporeIPH13PRO5G256LBL",
  "SingaporeIPH14PRM5G256PUR","SingaporeIPH14PRM5G256GLD","SingaporeIPH13PRM5G128LBL","MalaysiaIPH13PRM5G256LBL","SingaporeGXYZFLIP35G256GRN","SingaporeIPH14PRM5G128PUR",
  "SingaporeGXYZFLIP35G256PUR","SingaporeGXYZFLIP45G256BLU", "SingaporeIPH13PRM5G256GRY","SingaporeGXYZFLIP45G256GRP","SingaporeIPH13PRM5G256LBL",
  "SingaporeGXYZFLIP45G256PUR","SingaporeGXYS22U5G256BLK","SingaporeN20U5G256BRZ","SingaporeGXYS22U5G256BUR","MalaysiaGXYZFOLD35G256BLK","SingaporeGXYS21U5G256BLK",
  "SingaporeIPH14PRO5G256PUR","SingaporeGXYS22U5G256GRN"
])) & copy['ds'].isin(['2023-09-01','2023-10-01','2023-11-01'])].tail(50)

/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/430696852.py:6: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  ])) & copy['ds'].isin(['2023-09-01','2023-10-01','2023-11-01'])].tail(50)


,unique_id,ds,Country,SKU,XGB,y
2467,SingaporeGXYZFLIP35G256GRN,2023-10-01,Singapore,GXYZFLIP35G256GRN,8.163518,10
2468,SingaporeGXYZFLIP35G256GRN,2023-11-01,Singapore,GXYZFLIP35G256GRN,6.795429,15
2469,SingaporeGXYZFLIP35G256PUR,2023-09-01,Singapore,GXYZFLIP35G256PUR,9.604312,13
2470,SingaporeGXYZFLIP35G256PUR,2023-10-01,Singapore,GXYZFLIP35G256PUR,9.375942,13
2471,SingaporeGXYZFLIP35G256PUR,2023-11-01,Singapore,GXYZFLIP35G256PUR,7.801701,14
2484,SingaporeGXYZFLIP45G256BLU,2023-09-01,Singapore,GXYZFLIP45G256BLU,4.382067,9
2485,SingaporeGXYZFLIP45G256BLU,2023-10-01,Singapore,GXYZFLIP45G256BLU,5.760769,8
2486,SingaporeGXYZFLIP45G256BLU,2023-11-01,Singapore,GXYZFLIP45G256BLU,5.266486,8
2487,SingaporeGXYZFLIP45G256GRP,2023-09-01,Singapore,GXYZFLIP45G256GRP,5.251038,11
2488,SingaporeGXYZFLIP45G256GRP,2023-10-01,Singapore,GXYZFLIP45G256GRP,6.721947,9


In [104]:
# XGB
#future_df = copy.groupby('unique_id').filter(lambda x: x['y'].sum() >= forecast*10)
future_df = copy[copy['unique_id'].isin([
  "SingaporeIPH13PRO5G256GRY",
  "ThailandIPH13PRM5G256BLU",
  "SingaporeIPH14PRM5G256BLK",
  "SingaporeIPH13PRO5G256LBL",
  "SingaporeIPH14PRM5G256PUR",
  "SingaporeIPH14PRM5G256GLD",
  "SingaporeIPH13PRM5G128LBL",
  "MalaysiaIPH13PRM5G256LBL",
  "SingaporeGXYZFLIP35G256GRN",
  "SingaporeIPH14PRM5G128PUR",
  "SingaporeGXYZFLIP35G256PUR",
  "SingaporeGXYZFLIP45G256BLU",
  "SingaporeIPH13PRM5G256GRY",
  "SingaporeGXYZFLIP45G256GRP",
  "SingaporeIPH13PRM5G256LBL",
  "SingaporeGXYZFLIP45G256PUR",
  "SingaporeGXYS22U5G256BLK",
  "SingaporeN20U5G256BRZ",
  "SingaporeGXYS22U5G256BUR",
  "MalaysiaGXYZFOLD35G256BLK",
  "SingaporeGXYS21U5G256BLK",
  "SingaporeIPH14PRO5G256PUR",
  "SingaporeGXYS22U5G256GRN"
])]
future_df = future_df[future_df['ds'].isin(['2023-09-01','2023-10-01','2023-11-01'])]
eval_df = future_df.copy()
'''eval_df['unique_id'] = label_encoder.inverse_transform(eval_df['unique_id'])
ts_reshaped = eval_df['ds'].values.reshape(-1, 1)
eval_df['ds'] = ordinal_encoder.inverse_transform(ts_reshaped)'''
#price = df[['unique_id','Price']].copy().drop_duplicates()
#eval_df = eval_df.merge(price, left_on=['unique_id'], right_on=['unique_id'], how='left')
if train_ffc:
  dates = eval_df['ds'].unique()
  deval = dates[:-forward_forecast]
  eval_df = eval_df.query('ds in @deval')

if train_bfc:
  mse_xgb = mean_squared_error(eval_df['y'], eval_df['XGB'])
  rmse_xgb = np.sqrt(mse_xgb)
  r2_xgb = r2_score(eval_df['y'], eval_df['XGB'])

  if no_pre_xgb:
    # XGB No Predecessor Claims
    mse_no_pre = mean_squared_error(eval_df['y'], eval_df['XGB No Predecessor Claims'])
    rmse_no_pre = np.sqrt(mse_no_pre)
    r2_no_pre = r2_score(eval_df['y'], eval_df['XGB No Predecessor Claims'])


  # Print the results
print('XGB')
print('MSE:', mse_xgb, )
print('RMSE:', rmse_xgb,)
print('R2:', r2_xgb)
print()
if no_pre_xgb:
  print('XGB No Predecessor Demand')
  print('MSE:', mse_no_pre, )
  print('RMSE:', rmse_no_pre,)
  print('R2:', r2_no_pre)
  print()

XGB
MSE: 23.67623564505843
RMSE: 4.865823223778113
R2: 0.7527916498762601



/var/folders/27/kfjm3q7x1257nzn0tn92_pd00000gr/T/ipykernel_20748/2993283840.py:28: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  future_df = future_df[future_df['ds'].isin(['2023-09-01','2023-10-01','2023-11-01'])]


In [105]:
eval_df[eval_df['ds']=='2023-09-01']

,unique_id,ds,Country,SKU,XGB,y
573,MalaysiaGXYZFOLD35G256BLK,2023-09-01,Malaysia,GXYZFOLD35G256BLK,4.863064,6
1068,MalaysiaIPH13PRM5G256LBL,2023-09-01,Malaysia,IPH13PRM5G256LBL,9.016531,14
2187,SingaporeGXYS21U5G256BLK,2023-09-01,Singapore,GXYS21U5G256BLK,10.892229,13
2247,SingaporeGXYS22U5G256BLK,2023-09-01,Singapore,GXYS22U5G256BLK,11.477953,8
2250,SingaporeGXYS22U5G256BUR,2023-09-01,Singapore,GXYS22U5G256BUR,12.660853,18
2253,SingaporeGXYS22U5G256GRN,2023-09-01,Singapore,GXYS22U5G256GRN,12.915211,9
2466,SingaporeGXYZFLIP35G256GRN,2023-09-01,Singapore,GXYZFLIP35G256GRN,7.280852,12
2469,SingaporeGXYZFLIP35G256PUR,2023-09-01,Singapore,GXYZFLIP35G256PUR,9.604312,13
2484,SingaporeGXYZFLIP45G256BLU,2023-09-01,Singapore,GXYZFLIP45G256BLU,4.382067,9
2487,SingaporeGXYZFLIP45G256GRP,2023-09-01,Singapore,GXYZFLIP45G256GRP,5.251038,11
